# Advanced: Sending PatMatch output directly to Python 

The conclusion of the [previous notebook, 'PatMatch with more Python'](PatMatch with more Python.ipynb), alluded to the fact there was a way to further leverage the conputational space afforded by the Jupyter environment to skip over needing to save a file to actually pass results from shell scripts into Python. This notebook will demonstrate several of those.

## Preparing

Similar to the previous notebook, in order to insure everything is all set, act as if this is a new session in this Jupyter environment, and run the next cell so that you can step through the preparation steps to get a sequence file and prepare it. Plus, you'll get the file for script to convert it to dataframe and import its the main function. We don't need to run PatMatch yet because we will be addressing that part of the workflow in this notebook. 

Repeating these steps if you had already done so this session will cause no harm, and so go ahead and run this cell.

In [ ]:
!curl -O https://downloads.yeastgenome.org/sequence/S288C_reference/chromosomes/fasta/chrmt.fsa
!perl ../patmatch_1.2/unjustify_fasta.pl chrmt.fsa
!curl -O https://raw.githubusercontent.com/fomightez/sequencework/master/patmatch-utilities/patmatch_results_to_df.py
from patmatch_results_to_df import patmatch_results_to_df

## Running Patmatch and passing the results to Python without creating an output file intermediate

#### Option 1

First we'll do one of the several methods I have found to do this and show how to go to the next step entirely. This first example uses an approach illustrated [here](https://stackoverflow.com/a/42703609/8508004).  The result that is returned is of the type `IPython.utils.text.SList`, which offers some handy utility attributes associated with it as detailed [here](http://ipython.readthedocs.io/en/stable/api/generated/IPython.utils.text.html#IPython.utils.text.SList).

In [ ]:
output = !perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared 

That performed the matching step without generating an intermediate file. Next, run the next cell to combine this with use of main function of `patmatch_results_to_df.py` as illustrated in the [previous notebook, 'PatMatch with more Python'](PatMatch with more Python.ipynb).

In [ ]:
#patmatch_results_to_df.py with the `.n` attribute of IPython.utils.text.SList
my_pattern= "DDWDWTAWAAGTARTADDDD"
df = patmatch_results_to_df(output.n, pattern=my_pattern, name="promoter")

This option has the distinction that all the steps in it can be combined in one cell of the notebook. This is because the `!<command>` approach makes a temporary shell outside of the notebook environment to which it sends the command after the exclamation site. After that command is processed and any communication handled, that subshell where it ran is discarded. Given the transient nature of this environment you'll find `!cd` never seems to work as they discuss [here](http://ipython.readthedocs.io/en/stable/interactive/magics.html), search `!cd` to find the pertinent section where they also show you the line magics solution. ([Here](https://jakevdp.github.io/PythonDataScienceHandbook/01.05-ipython-and-shell-commands.html)is another good resource in this regard.)

The additional options for passing the results demonstrated below instead rely on cell magics, and so the output needs to be captured from a cell before the next steps can be undertaken in an additional cell. While not a big deal that extra cells are involved, I find the `!<command>` approach can be nice for streamlining things when making mini-pipelines/workflows using the Jupyter environment as a 'glue' to merge Python and command line use. 

The additional options for passing the results will be stepped through in a manner similar to 'Option 1' using different approaches for step 1 each time and step #2 varied to handle as necessary.

#### Option 2

In this option, [`%%capture` cell magics](http://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-capture) is used, and then using the attributes of the `utils.cpature` object you can easily get the stdout and/or stderr as a string, see [here]( http://ipython.readthedocs.io/en/stable/api/generated/IPython.utils.capture.html).

In [ ]:
%%capture out
!perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared

In [ ]:
#patmatch_results_to_df.py with the `.stdout` attribute of `utils.cpature`
my_pattern= "DDWDWTAWAAGTARTADDDD"
df = patmatch_results_to_df(out.stdout, pattern=my_pattern, name="promoter")

#### Option 3

In this option, a varation of `%%bash` cell magic is used to send the output to a variable as illustrated [here](https://stackoverflow.com/a/24776049/8508004).

In [ ]:
%%bash --out output
perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared

In [ ]:
#patmatch_results_to_df.py by passing the stdout to a varible using %%bash cell magic
my_pattern= "DDWDWTAWAAGTARTADDDD"
df = patmatch_results_to_df(output, pattern=my_pattern, name="promoter")

#### Option 4 (<font color='red'>** NEVERMIND, THIS DOESN'T SEEM TO WORK.**</font>** NEVERMIND, THIS DOESN'T SEEM TO WORK.**)

In this option, IPython's output caching system as summarized [here](https://stackoverflow.com/a/27952661/8508004) is used. Step#1 requires nothing different than the normal command that sends the output to the output stream of the cell. The trick comes in step#2 where that output is used by pointing to it with an underscore. <font color='red'>** DESPITE THE FACT I HAD NOTED THIS APPROACH WORKED ONCE IN THE PAST WITH A RESULT FROM THE SHELL, IT DOESN'T SEEM TO WORK AND MAYBE I WAS FOOLED BY SOMETHING IN THE NAMESPACE?!?**</font>

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared 

In [ ]:
#patmatch_results_to_df.py by reference the output of the previous cell using IPython's 'output caching system'
my_pattern= "DDWDWTAWAAGTARTADDDD"
df = patmatch_results_to_df(_, pattern=my_pattern, name="promoter")